# **Install & Import**

In [3]:
# Install required packages (run only once)
!pip install -q gradio transformers Pillow

import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np
import gradio as gr
import requests
from PIL import Image, ImageDraw, ImageFont
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from transformers import Mask2FormerImageProcessor, Mask2FormerForUniversalSegmentation

# Load Models

In [4]:
# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load models
print("Loading segmentation model...")
seg_processor = Mask2FormerImageProcessor.from_pretrained("facebook/mask2former-swin-large-coco-panoptic")
seg_model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-coco-panoptic").to(device)

print("Loading captioning model...")
caption_processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
caption_model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

print("Models loaded successfully!")

Using device: cpu
Loading segmentation model...


preprocessor_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/866M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/866M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Models loaded successfully!


# Utility Functions

In [8]:
# COCO panoptic classes (91 classes)
CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse",
    "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie",
    "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon",
    "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut",
    "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book",
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]
COLORS = [
    "#" + ''.join([np.random.choice(list('0123456789ABCDEF')) for _ in range(6)])
    for _ in range(len(CLASSES))
]

def download_image(url):
    """Download image from URL"""
    try:
        response = requests.get(url, stream=True, timeout=10)
        response.raise_for_status()
        return Image.open(response.raw).convert("RGB")
    except Exception as e:
        print(f"Error downloading image: {e}")
        return None

def process_segmentation(image):
    """Perform panoptic segmentation and return masks with labels"""
    try:
        inputs = seg_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = seg_model(**inputs)
        processed_sizes = [(image.size[1], image.size[0])]
        result = seg_processor.post_process_panoptic_segmentation(
            outputs, target_sizes=processed_sizes
        )[0]
        masks, labels, colors = [], [], []
        for segment in result["segments_info"]:
            label_id = segment["label_id"]
            mask = (result["segmentation"] == segment["id"]).cpu().numpy()
            if label_id < len(CLASSES):
                masks.append(mask)
                labels.append(CLASSES[label_id])
                colors.append(COLORS[label_id])
        return masks, labels, colors
    except Exception as e:
        print(f"Segmentation error: {e}")
        return [], [], []

def generate_caption(image, objects):
    """Generate contextual caption mentioning detected objects"""
    try:
        prompt = f"Question: Describe this image including these objects: {', '.join(objects[:5])}. Answer:"
        inputs = caption_processor(
            images=image,
            text=prompt,
            return_tensors="pt"
        ).to(device, torch.float16 if device == "cuda" else torch.float32)
        generated_ids = caption_model.generate(
            **inputs,
            max_new_tokens=100
        )
        caption = caption_processor.batch_decode(
            generated_ids,
            skip_special_tokens=True
        )[0].strip()
        return caption
    except Exception as e:
        print(f"Captioning error: {e}")
        return "Could not generate caption"

def visualize_results(image, caption, masks, labels, colors):
    """Create output visualization with segmentation and caption"""
    try:
        if not isinstance(image, Image.Image):
            image = Image.fromarray(image)
        output_image = image.copy().convert("RGBA")
        draw = ImageDraw.Draw(output_image)
        overlay = Image.new("RGBA", output_image.size, (0, 0, 0, 0))
        overlay_draw = ImageDraw.Draw(overlay)
        for mask, color_str in zip(masks, colors):
            color = tuple(int(color_str.lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
            y, x = np.where(mask)
            if len(x) > 0 and len(y) > 0:
                bbox = [min(x), min(y), max(x), max(y)]
                overlay_draw.rectangle(bbox, outline=color + (200,), width=3)
        output_image = Image.alpha_composite(output_image, overlay)
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()
        # Use textbbox instead of textsize
        text_bbox = draw.textbbox((0, 0), caption, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        draw.rectangle(
            [(10, 10), (text_width + 20, text_height + 20)],
            fill=(0, 0, 0, 180),
            outline=(255, 255, 255, 180)
        )
        draw.text((15, 15), caption, font=font, fill=(255, 255, 255, 255))
        legend_y = 50
        for label, color_str in zip(labels, colors):
            color = tuple(int(color_str.lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
            draw.rectangle([(10, legend_y), (30, legend_y + 20)], fill=color + (200,))
            draw.text((35, legend_y), label, font=font, fill=(255, 255, 255, 255))
            legend_y += 25
        return output_image
    except Exception as e:
        print(f"Visualization error: {e}")
        return image

## Main Pipeline

In [9]:
def process_image(input_image, image_url=""):
    """Main processing pipeline"""
    try:
        # Handle URL or uploaded image
        if image_url and isinstance(image_url, str) and image_url.strip():
            image = download_image(image_url)
            if image is None:
                return None, "Failed to download image"
        elif input_image is not None:
            if isinstance(input_image, Image.Image):
                image = input_image.convert("RGB")
            else:
                return None, "Invalid image input"
        else:
            return None, "No image provided"
        masks, labels, colors = process_segmentation(image)
        if not masks:
            return image, "No objects detected"
        caption = generate_caption(image, labels)
        output_image = visualize_results(image, caption, masks, labels, colors)
        return output_image, caption
    except Exception as e:
        print(f"Processing error: {e}")
        return None, f"Error: {str(e)}"

## Gradio Interface

In [ ]:
with gr.Blocks(title="Image Captioning & Segmentation") as demo:
    gr.Markdown("# 🖼️ Context-Aware Image Understanding")
    gr.Markdown("Combines panoptic segmentation with contextual captioning")

    with gr.Row():
        with gr.Column():
            input_type = gr.Radio(
                ["Upload", "URL"],
                label="Input Type",
                value="Upload"
            )
            image_input = gr.Image(type="pil", label="Upload Image")
            url_input = gr.Textbox(
                label="Image URL",
                placeholder="https://example.com/image.jpg",
                visible=False
            )

            def toggle_input(input_type):
                return {
                    image_input: gr.update(visible=input_type == "Upload"),
                    url_input: gr.update(visible=input_type == "URL")
                }

            input_type.change(
                toggle_input,
                inputs=input_type,
                outputs=[image_input, url_input]
            )

            process_btn = gr.Button("Process Image", variant="primary")

        with gr.Column():
            output_image = gr.Image(label="Result", interactive=False)
            caption_output = gr.Textbox(label="Generated Caption")

    gr.Markdown("## Example Images")
    examples = gr.Examples(
        examples=[
            ["https://picfiles.alphacoders.com/280/thumb-1920-280162.jpg"],
            ["https://wallpapers-all.com/uploads/posts/2016-11/1_dog.jpg"],
            ["https://bikeadvice.in/wp-content/uploads/2021/11/pulsar-f250deliveries-pic.jpg"]
        ],
        inputs=[url_input],
        outputs=[output_image, caption_output],
        fn=process_image,
        cache_examples=False  # Disable caching to avoid JSON errors
    )

    process_btn.click(
        fn=process_image,
        inputs=[image_input, url_input],
        outputs=[output_image, caption_output]
    )

print("Launching Gradio interface...")
demo.launch(share=True, debug=True)

Launching Gradio interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://133d62f12b607e6598.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
`label_ids_to_fuse` unset. No instance will be fused.
